# Youbike 2.0 Formulation Modeling

In [5]:
# !pip install gurobipy

# from google.colab import drive
# drive.mount('/content/drive')

from gurobipy import *

## load testcases

In [2]:
import joblib
prefix = './data/testcases'
testcase = joblib.load(f'{prefix}/testcase_even_k5_Q30')
n_node = testcase['n_node']
d = testcase['d']
b = testcase['b']
Q = testcase['Q']
n_vehicle = testcase['n_vehicle']

In [23]:
VRP()

GurobiError: Unsupported type (<class 'generator'>) for LinExpr addition argument

In [22]:
def VRP(n_node = n_node, 
        n_vehicle = n_vehicle, 
        d = d, Q = Q, b = b):
    """
    參數定義：
    n_node - 整數，租借站個數（不包含depot node）
    n_vehicle - 整數，卡車數量
    d　- 二維list，大小為n_node + 1 * n_node + 1，d[i][j]為node i到node j的距離，node 0到所有node距離為0
    b - 一維list，大小為n_node + 1, b[j]為node j的淨供給，b[0] = -(所有node總和)
    Q - 整數，一輛卡車能承載的腳踏車數
    """
    vrp = Model("vrp")

    # add variables as a list
    x = []
    r = []
    for k in range(n_vehicle):
        x.append([])
        r.append([])
        for i in range(n_node + 1):
            x[k].append([])
            r[k].append([])
            for j in range(n_node + 1):
                x[k][i].append(vrp.addVar(lb=0, vtype = GRB.BINARY, name='_'.join(["x", str(i), str(j), str(k + 1)])))
                r[k][i].append(vrp.addVar(lb=0, ub=Q, vtype = GRB.CONTINUOUS, name='_'.join(["x", str(i), str(j), str(k + 1)])))
    
    w = vrp.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

    # set obj and constraint
    vrp.setObjective(w, GRB.MINIMIZE)

    for k in range(n_vehicle):
        for i in range(n_node + 1):
            for j in range(n_node + 1):
                if i == j:
                    vrp.addConstr(x[k][i][j] == 0, '{} != {} constraint for vehicle {}'.format(i, j, k + 1))
    
    # 進=出
    for j in range(n_node + 1):
        for k in range(n_vehicle):
            vrp.addConstr(quicksum(x[k][i][j] for i in range(n_node + 1))
                             == quicksum(x[k][j][i] for i in range(n_node + 1)),'in=out')
    
    # 起點&終點為0
    vrp.addConstrs((quicksum(x[k][0][j] for j in range(n_node + 1)) == 1 for k in range(n_vehicle)), 'start')
    vrp.addConstrs((quicksum(x[k][j][0] for j in range(n_node + 1)) == 1 for k in range(n_vehicle)), 'end')

    # 連結r和x，Q為r的upperbound
    for j in range(n_node + 1):
        for k in range(n_vehicle):
            vrp.addConstrs((r[k][i][j] <= Q * x[k][i][j] for i in range(n_node + 1)), 'connect r and x')

    
    
    # demand fulfillment
    vrp.addConstrs((quicksum((r[k][i][j] for i in range(n_node + 1)) 
                             for k in range(n_vehicle)) + b[j] == 
                    quicksum((r[k][j][i] for i in range(n_node + 1)) 
                             for k in range(n_vehicle)) for j in range(n_node + 1)), 'demand fulfillment')

    # 載0台出門
    vrp.addConstrs((quicksum(r[k][0][j] for j in range(n_node + 1)) == 0 for k in range(n_vehicle)), 'empty at start')

    # 吸收多餘supply
    vrp.addConstr(quicksum((r[k][j][0] for j in range(n_node + 1)) for k in range(n_vehicle)) == quicksum(b[j] for j in range(1, n_node + 1)), 'remaining supply')

    # makespan
    vrp.addConstrs((w >= quicksum((d[i][j] * x[k][i][j] for i in range(1, n_node + 1)) for j in range(1, n_node + 1)) for k in range(n_vehicle)), 'makespan')

    # subtour還沒想到

    vrp.optimize()

    return x, r
